In [24]:
from simclr import SimCLR
import torch
import numpy as np
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
from flash.core.optimizers import LARS


from torch.utils.tensorboard import SummaryWriter
from simclr.modules.transformations import TransformsSimCLR
from simclr.modules import NT_Xent


In [3]:
image_size = 224
root = 'datasets/'

dataset = CIFAR10(root=root, download=True, transform = TransformsSimCLR(size = image_size))
torch.manual_seed(43)
batch_size = 128
train_loader = DataLoader(dataset, 
                          batch_size, 
                          shuffle=False,
                          drop_last = True,
                          num_workers=2,
                          sampler = None)

Files already downloaded and verified


In [4]:
encoder = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False) 
projection_dim = 64
n_features = encoder.fc.in_features  # get dimensions of last fully-connected layer
model = SimCLR(encoder, projection_dim, n_features)

Using cache found in C:\Users\Shulu/.cache\torch\hub\pytorch_vision_v0.10.0


In [17]:
optimizer = LARS(model.parameters(), lr = 0.075 * np.sqrt(batch_size), weight_decay = 1e-6)
criterion = NT_Xent(batch_size, temperature = 0.2, world_size=1)

In [6]:
train_dataset = CIFAR10(root = 'datasets/', transform = transforms.ToTensor())

loader = DataLoader(train_dataset, 
                          batch_size, 
                          shuffle=False,
                          drop_last = True,
                          num_workers=2,
                          sampler = None)

In [25]:
writer = SummaryWriter()

In [ ]:
def train(global_step, train_loader, model, criterion, optimizer, writer):
    loss_epoch = 0
    for steps, (i, _) in enumerate(train_loader):
    
        optimizer.zero_grad()
        i = i.cuda(non_blocking = True)
        j = i.cuda(non_blocking = True)
        h_i, h_j, z_i, z_j = model(i, j)
        loss = criterion(z_i, z_j)
        loss.backward()
        optimizer.step()

        if steps % 50 == 0:
            print(f"Step [{steps}/{len(train_loader)}]\t Loss: {loss.item()}")

        writer.add_scalar("Loss/train_epoch", loss.item(), global_step)
        loss_epoch += loss.item()
        global_step += 1
    return loss_epoch

global_step = 0
current_epoch = 0
epochs = 50

from tqdm import tqdm as tqdm
for epoch in tqdm(range(epochs)):
    loss_epoch = train(global_step, train_loader, model, criterion, optimizer, writer)

    writer.add_scalar("Loss/train", loss_epoch / len(loader), epochs)
    print(
        f"Epoch [{current_epoch}/{epoch}]\t Loss: {loss_epoch / len(train_loader)}\t"
    )
    current_epoch += 1

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

In [ ]:
loss_epoch = 0

#the transformed dataset, my PC gets stuck inside the for-statement
#Currently working with the for-loop above for debugging and testing purposes //Mohamad

for step, ((x_i, x_j), _) in enumerate(train_loader): #program gets stuck exactly here
    optimizer.zero_grad()
    x_i = x_i.cuda(non_blocking=True)
    x_j = x_j.cuda(non_blocking=True)

    # positive pair, with encoding
    h_i, h_j, z_i, z_j = model(x_i, x_j)

    loss = criterion(z_i, z_j)
    loss.backward()

    optimizer.step()

